In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
from tqdm import tqdm
import itertools

# Importing Pauli matrices
from pauli_matrices.pauli_matrices import I, X, Y, Z

# Importing functions
from pauli_matrices.pauli_chain import get_Pauli_chain
from hamiltonian.hamiltonian import get_hamiltonian
from Bell_operator.Bell_operator import get_Bell_term
from classical_optimization.classical_optimization import classical_optimization

In [2]:
# Defining qubit transform
qubit_transform = 'JW'

# Obtainig Hamiltonian
hamiltonian = get_hamiltonian(qubit_transform = qubit_transform)

# Extracting parameters
H = hamiltonian.matrix_form
N = hamiltonian.N

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# Extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))

E_G

-1.8510456784448643

In [3]:
# Defining measurements of the parties
M1 = M2 = M3 = M4 = [I, X, Y, Z]

# Consrtucting a measurement matrix
M = np.array([
    M1,
    M2,
    M3,
    M4
])

# Initializing the Bell operator, the Bell inequality and a string containing the variables
B = 0
var = []
var_dict = {}

# Creating indices for all measurements
temp_indices = list(itertools.product([i for i in range(len(M1))], repeat=(N)))

# Checking for one and two body terms requirement
indices = []
one_and_two_body = False

if one_and_two_body:
    for j in temp_indices:
        if np.sum([j[i] == 0 for i in range(len(j))]) >= 2:
            indices.append(j)
else:
    indices = temp_indices.copy()

# Calculating the Bell operator
for measurement_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [M[p, measurement_indices[p]] for p in range(N)]

    # Calculating the Bell terms and the corresponding variable
    B_temp, var_name, var_temp = get_Bell_term(measurement_choices, measurement_indices)
    
    # Adding and storing the values
    B += B_temp
    var.append(var_temp)
    var_dict[var_name] = var_temp 

100%|██████████| 256/256 [00:04<00:00, 53.93it/s]


In [4]:
# Obtaining the sytsem of equations (this takes ages)
pauli_matrices = [I, X, Y, Z]

# Initializing the equations list
eqs = []

# Calculating new set of indices
indices = list(itertools.product([i for i in range(len(pauli_matrices))], repeat=N))

# Looping over the equations
for projector_indices in tqdm(indices):

    # Constructing the measurement choices
    measurement_choices = [pauli_matrices[p] for p in projector_indices]
    Projector = get_Pauli_chain(measurement_choices)

    # calculating system of equations
    eqs.append( 
        Eq( 
            np.trace( np.matmul(B, Projector)).simplify(), 
            np.trace( np.matmul(H, Projector))
        ) 
    )
   
# Solving the system of equations
ans = solve(eqs, var)

# Updating the dictionary
for key in ans.keys():
    var_dict[str(key)] = ans[key]

100%|██████████| 256/256 [03:09<00:00,  1.35it/s]


In [6]:
# Obtaining the Bell operator 
for i in tqdm(range(2**N)):
    for j in range(2**N):
        for variable in var:
            B[i,j] = B[i,j].subs(variable, ans[variable])

# Calculting the quantum value
beta_Q = np.matmul(psi_G_dagger, np.matmul(B, psi_G))
beta_Q

100%|██████████| 16/16 [00:09<00:00,  1.68it/s]


-1.85104567844486

In [5]:
# Defining the number of possible measurements
m = len(M1)-1

# Obtaining the classical bound with the recursive algorithm
beta_C_rec = classical_optimization(var_dict, N, m)

# Obtaining the classical bound by comparing all possible configurations
beta_C = classical_optimization(var_dict, N, m, recursive=False)

# Printing the results
print(beta_C_rec, beta_C)

100%|██████████| 4096/4096 [00:54<00:00, 75.46it/s]

1.35509750000000 2.01174800000000


In [7]:
def calc_inequality(M, var_dict, indices):
    # Extracting the coefficients and the matrix products
    coeffs = []
    M_elements = []

    for idx in indices:
        # Initializing coefficients and matrix products
        name = ''
        M_prod = 1

        # Looping over the m measurements
        for j in range(len(idx)):
            name += str(idx[j])
            M_prod *= M[idx[j], j]

        # Storing values
        coeffs.append(var_dict['c_{'+name+'}'])
        M_elements.append(M_prod)
    
    # Calculating the Bell inequality
    return np.sum([coeffs[j]*M_elements[j] for j in range(len(indices))])

In [116]:
# Initializing the matrix M, 0'th row does not correspond to a measurement
M_c = np.ones((m+1, N))   # DOes this need to be initialized in a random manner?
# M_c[1:, :] = np.random.randint(2, size=(m, N))*2-1

# Initializing temporary indices
temp_indices = list(itertools.product([i for i in range(m+1)], repeat=N))

# Initializing E_i
E_i = 0

# Calculating the possible outcomes for the i'th row of the matrix M
possible_configurations = list(itertools.product([+1, -1], repeat=m))

# Calculating the i'th indices
for idx in tqdm(range(N)):

    # This needs to be done for each iteration of idx, the above does not have to be done each iteration
    indices = []

    # First condition ensures that we do not have any double counting
    # Second condition ensures that we do not take a constant non measurable value into account
    # Last condition makes it so we do not capture any measurements from other observables
    for j in temp_indices:

        if np.sum(j[:idx]) == 0 and np.sum(j) != 0 and j[idx] != 0: 
            indices.append([*[0] * (idx), *j[idx:]])

    # Updating correlation matrix and setting a very high value for h_i such that we always start with a lowest configuration after one iteration
    M_c_temp = M_c.copy()
    h_i = 1e6

    # Looping over the different configurations
    for conf in possible_configurations:

        # Updating the correlation matrix
        M_c_temp[1:, idx] = conf


        # Calculating h_i
        # Extracting the coefficients and the matrix products
        coeffs = []
        M_elements = []

        for idx2 in indices:
            # Initializing coefficients and matrix products
            name = ''
            M_prod = 1

            # Looping over the m measurements
            for j in range(len(idx2)):
                name += str(idx2[j])
                M_prod *= M_c_temp[idx2[j], j]

            # Storing values
            coeffs.append(var_dict['c_{'+name+'}'])
            M_elements.append(M_prod)

        h_i_temp = np.sum([coeffs[j]*M_elements[j] for j in range(len(indices))])
        
        # Checking if the new h_i is smaller than the old one, if yes, then we adapt the new value
        if h_i_temp < h_i:
            M_c = M_c_temp.copy()
            h_i = h_i_temp


    # Adding the last configuration of h_i to E_i
    E_i += h_i

# Obtaining the classical bound
beta_C_rec = -E_i
print(beta_C_rec)

100%|██████████| 4/4 [00:00<00:00, 32.52it/s]

1.35509750000000


In [120]:
idx2

[0, 0, 0, 3]

In [119]:
# This needs to be done for each iteration of idx, the above does not have to be done each iteration
indices = []
idx = 3
# First condition ensures that we do not have any double counting
# Second condition ensures that we do not take a constant non measurable value into account
# Last condition makes it so we do not capture any measurements from other observables
for j in temp_indices:

    if np.sum(j[:idx]) == 0 and np.sum(j) != 0 and j[idx] != 0: 
        indices.append([*[0] * (idx), *j[idx:]])
indices

[[0, 0, 0, 1], [0, 0, 0, 2], [0, 0, 0, 3]]

In [110]:
# Initializing the matrix M, 0'th row does not correspond to a measurement
M_c = np.ones((m+1, N))   # DOes this need to be initialized in a random manner?

# Initializing temporary indices
temp_indices = list(itertools.product([i for i in range(m+1)], repeat=N))

# Obtaining all possible indices and configurations
indices = temp_indices[1:] # exlcuduing the identity term
possible_configurations = list(itertools.product([-1, +1], repeat=m*N))

# Initializing Bell inequality
I = 1e6

for conf in tqdm(possible_configurations):
    # Updating correlation matrix
    M_c[1:, :] = np.reshape(conf, (m, N))

    # Calculating the Bell inequality
    I_new = calc_inequality(M_c, var_dict, indices)

    # Checking if we have a new minimum
    if I_new < I:
        I = I_new
        M_c2 = M_c.copy()

beta_C_all = -I

100%|██████████| 4096/4096 [00:46<00:00, 87.50it/s]


In [112]:
indices

[(0, 0, 0, 1),
 (0, 0, 0, 2),
 (0, 0, 0, 3),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 2),
 (0, 0, 1, 3),
 (0, 0, 2, 0),
 (0, 0, 2, 1),
 (0, 0, 2, 2),
 (0, 0, 2, 3),
 (0, 0, 3, 0),
 (0, 0, 3, 1),
 (0, 0, 3, 2),
 (0, 0, 3, 3),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 2),
 (0, 1, 0, 3),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 2),
 (0, 1, 1, 3),
 (0, 1, 2, 0),
 (0, 1, 2, 1),
 (0, 1, 2, 2),
 (0, 1, 2, 3),
 (0, 1, 3, 0),
 (0, 1, 3, 1),
 (0, 1, 3, 2),
 (0, 1, 3, 3),
 (0, 2, 0, 0),
 (0, 2, 0, 1),
 (0, 2, 0, 2),
 (0, 2, 0, 3),
 (0, 2, 1, 0),
 (0, 2, 1, 1),
 (0, 2, 1, 2),
 (0, 2, 1, 3),
 (0, 2, 2, 0),
 (0, 2, 2, 1),
 (0, 2, 2, 2),
 (0, 2, 2, 3),
 (0, 2, 3, 0),
 (0, 2, 3, 1),
 (0, 2, 3, 2),
 (0, 2, 3, 3),
 (0, 3, 0, 0),
 (0, 3, 0, 1),
 (0, 3, 0, 2),
 (0, 3, 0, 3),
 (0, 3, 1, 0),
 (0, 3, 1, 1),
 (0, 3, 1, 2),
 (0, 3, 1, 3),
 (0, 3, 2, 0),
 (0, 3, 2, 1),
 (0, 3, 2, 2),
 (0, 3, 2, 3),
 (0, 3, 3, 0),
 (0, 3, 3, 1),
 (0, 3, 3, 2),
 (0, 3, 3, 3),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 2),
 (1, 0, 0,